<a href="https://colab.research.google.com/github/squeeko/TensorflowProjects/blob/master/Tensorflow2_QuickStart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

Load and prepare the MNIST dataset. Convert the samples from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


Build the tf.keras.Sequential model by stacking layers. Choose an optimizer and loss function for training:

In [3]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Flatten(input_shape=(28, 28)),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.58306307, -0.70052946, -0.02297604,  0.5340719 ,  0.06329438,
        -0.24514839,  0.70404416,  0.05417342,  0.26297706, -0.33305806]],
      dtype=float32)

The tf.nn.softmax function converts these logits to "probabilities" for each class:

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.05226121, 0.04646912, 0.09150022, 0.15971474, 0.09974449,
        0.07327131, 0.18930541, 0.09883887, 0.12178954, 0.06710505]],
      dtype=float32)

**Note:** It is possible to bake this tf.nn.softmax in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

The ***losses.SparseCategoricalCrossentropy*** loss takes a vector of logits and a True index and returns a scalar loss for each example.

In [9]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.log(1/10) ~= 2.6.

In [12]:
loss_fn(y_train[:1], predictions).numpy()

2.6135862

In [13]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The Model.fit method adjusts the model parameters to minimize the loss:

In [14]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2930 - accuracy: 0.9158
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1441 - accuracy: 0.9568
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1079 - accuracy: 0.9673
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0882 - accuracy: 0.9727
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0739 - accuracy: 0.9766


The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [15]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0770 - accuracy: 0.9775


[0.07695811241865158, 0.9775000214576721]

The image classifier is now trained to ~98% accuracy on this dataset. 

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [16]:
probability_model = tf.keras.Sequential([
                                         model,
                                         tf.keras.layers.Softmax()
])

In [17]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.6364760e-06, 7.7265003e-09, 1.4079367e-05, 1.4339946e-04,
        2.8003952e-11, 4.2901490e-08, 2.5099730e-13, 9.9983788e-01,
        4.6927596e-07, 1.5126269e-06],
       [1.2419268e-10, 1.0290239e-05, 9.9998856e-01, 1.0004338e-06,
        1.5415449e-15, 7.9716711e-10, 4.1797494e-09, 4.7296412e-13,
        9.8904017e-08, 2.8735324e-15],
       [2.7101951e-08, 9.9882585e-01, 4.1217598e-05, 6.0600161e-07,
        1.7146227e-06, 9.4931335e-07, 6.6033272e-07, 1.0399391e-03,
        8.8962115e-05, 8.8257131e-08],
       [9.9989319e-01, 2.7078645e-08, 3.7031172e-05, 3.1814618e-06,
        1.6029771e-06, 3.5413013e-05, 2.3772884e-05, 2.2389825e-06,
        4.3156238e-08, 3.5396554e-06],
       [5.8534733e-06, 2.8482774e-10, 2.1669330e-05, 5.2961667e-08,
        9.9952602e-01, 5.5673024e-07, 1.3114072e-06, 4.4961876e-06,
        2.7019760e-07, 4.3973941e-04]], dtype=float32)>